# Deterministic Uncertainty Sets

In [ ]:
import cvxpy as cp
import scipy as sc
from sklearn import datasets
import numpy as np
import numpy.random as npr
import numpy.testing as npt
import torch
import pandas as pd
import lro
import matplotlib.pyplot as plt
RTOL = 1e-04
ATOL = 1e-04
import warnings
warnings.filterwarnings("ignore")
plt.rcParams.update({
    "text.usetex":True,
    "font.size":18,
    "font.family": "serif"
})
colors = ["tab:blue", "tab:green", "tab:orange", 
          "tab:red", "tab:purple", "tab:brown", "tab:pink", "tab:grey", "tab:olive"]

### Formulating the uncertainty set

The first step is to formulate the uncertainty set. There are multiple options, with their associated input parameters. The uncertainty set can be formulated within the uncertain parameter, which takes in the size of the uncertainty and an uncertainty set as parameters.

Box uncertainty: $\{ u \mid\| u \|_\infty \leq \rho \}$ 

- $\rho$ : float, optional  
  * Box scaling. Default 1.0.
- center : np.array, optional  
  * An array of dimension n, n the dimension of the uncertanty.
        Translation of the center of the box. If lengths is passed but
        not center it defaults to an array of zeros.
- side : np.array, optional  
  * An array of dimension n, n the dimension of the uncertanty.
        Length of each side of the box. If center is passed but side not
        it defaults to an array of 2.


In [2]:
# size of uncertain parameter
m = 5

# Box uncertainty set example
box_u = lro.UncertainParameter(m, 
                              uncertainty_set = lro.Box(center=np.zeros(m),
                                                        side=np.ones(m), 
                                                        rho=2.))



Ellipsoidal uncertainty: $\{u \mid \| u \|_p \leq \rho \}$ 
- $\rho$ : float, optional  
  * Ellipsoid scaling. Default 1.0. 
- $p$ : integer, optional  
  * Order of the norm. Default 2.

In [3]:
# Ellipsoidal uncertainty set example
ellip_u = lro.UncertainParameter(m,
                                uncertainty_set = lro.Ellipsoidal(p = 2, 
                                                                  rho=2.))

Budget uncertainty: $\{u \mid \| u \|_\infty \leq \rho_1, \| u \|_1 \leq \rho_2\}$ 

- $\rho_1$ : float, optional  
  * Box scaling. Default 1.0. 
- $\rho_2$ : float, optional
  * 1-norm scaling. Default 1.0.



In [4]:
# Budget uncertainty set example
budget_u = lro.UncertainParameter(m, 
                                  uncertainty_set = lro.Budget(rho1=2., 
                                                               rho2 = 1.))

Polyhedral uncertainty: $\{ u \mid Du \leq d\}$

- $D$ : np.array  
- $d$ : np.array  

In [5]:
# Polyhedral uncertainty set example
poly_u = lro.UncertainParameter(m, 
                                uncertainty_set = lro.Polyhedral(
                                                        D = np.ones((3,m)), 
                                                        d = np.array([1,2,3])))

###  Formulating the Robust Problem

We can now fomulate the Robust Problem, treating the uncertain parameter as a regular parameter.

#### Example 1: Affine transformed LP, ellipsoidal uncertainty
We solve the problem
$$
\begin{array}{ll}
\text{minimize} & c^Tx\\
\text{subject to}  & (Pu+a)^Tx \leq  10,\\
\end{array}
$$
where $c$, $P$, and $a$ are constants, and $u$ is the uncertain parameter from the ellipsoidal set above.

In [6]:
#restate the ellipsoidal set
ellip_u = lro.UncertainParameter(m,
                                uncertainty_set = lro.Ellipsoidal(p = 2, 
                                                                  rho=2., A = np.eye(m)))
n = 4

# formulate cvxpy variable
x_r = cp.Variable(4)

# formulate problem constants
P = 3. * np.eye(m)[:n, :]
a = 0.1 * np.random.rand(n)
c = np.random.rand(n)

# formulate objective
objective = cp.Minimize(c@x_r)

# formulate constraints
constraints = [(P@ellip_u +a)@ x_r <= 10]

# formulate Robust Problem
prob_robust = lro.RobustProblem(objective, constraints)

# solve
new_prob = prob_robust.dualize()
new_prob.solve()
print("LRO objective value: ", new_prob.objective.value, "\nLRO x: ", x_r.value)


LRO objective value:  -2.2220977010172613 
LRO x:  [-0.99824674 -1.0170179  -0.38059305 -0.84866837]


We compare the above with its explicit reformulation and solution
$$
\begin{array}{ll}
\text{minimize} & c^Tx\\
\text{subject to}  & a^Tx + \rho\|P^Tx\|_2 \leq  10,\\
\end{array}
$$

In [7]:
# formulate cvxpy variable
x = cp.Variable(4)

# formulate objective
objective = cp.Minimize(c@x)

# formulate constraints
constraints = [a@x + 2*cp.norm(P.T@x,2) <= 10]

# formulate problem
prob_cvxpy = cp.Problem(objective, constraints)

# solve
prob_cvxpy.solve()

print("Cvxpy objective value: ", prob_cvxpy.objective.value, "\nCvxpy x: ", x.value)

# assert x values are equal
npt.assert_allclose(x_r.value, x.value, rtol=RTOL, atol=ATOL)


Cvxpy objective value:  -2.2220976975136297 
Cvxpy x:  [-0.99824352 -1.01701471 -0.38060504 -0.84867089]


We see that they provide the same solution.

#### Example 2: Multiple uncertain terms, box uncertainty
We solve the problem
$$
\begin{array}{ll}
\text{minimize} & c^Tx\\
\text{subject to}  & (Pu)^Tx  \leq  10 + 2u^Tx,\\
& x \geq 0,\quad x \leq 1,
\end{array}
$$
where again $c$, $P$, and $a$ are constants, and $u$ is the uncertain parameter from a box uncertainty set. We note that the uncertain parameter can appear more than once in the uncertain constraint.  

The box uncertainty set is indentical to an affine transformed ellipsoidal uncertainty set. If the center of the box is $b$, and side is $s$, formulating the box uncertainaty set is equivalent to formulating an ellipsoidal uncertainty set with $p = \infty$ and an affine transform $\tilde{u} = \mathbf{diag}\{1/2s\}u + b = Au + b$.

With ellipsoidal uncertainty, the above problem is then 
$$
\begin{array}{ll}
\text{minimize} & c^Tx\\
\text{subject to}  & (PAu+ Pb)^Tx  \leq  10 + (2Au + 2b)^Tx\\
& x \geq 0,\quad x \leq 1,
\end{array}
$$

In [8]:
#formulate using the box set
center = 0.5*np.ones(m)
side = 0.1*np.array([1,2,3,4,5])
box_u = lro.UncertainParameter(m, 
                              uncertainty_set = lro.Box(center=center,
                                                        side=side, 
                                                        rho=2., A = np.eye(m)))
n = 5

# formulate cvxpy variable
x_box = cp.Variable(n)

# formulate problem constants
P = 3*np.random.rand(n,m)
c = np.random.rand(n)

# formulate objective
objective = cp.Minimize(-c@x_box)

# formulate constraints
constraints = [P@box_u@ x_box - 2*box_u@x_box <= 10 , x_box >=0, x_box<=1]

# formulate Robust Problem
prob_robust = lro.RobustProblem(objective, constraints)

# solve
newprob = prob_robust.dualize()
newprob.solve()
print("Box objective value: ", newprob.objective.value, "\nBox x: ", x_box.value)

#formulate using the ellipsoidal set
ellip_u = lro.UncertainParameter(m,
                                uncertainty_set = lro.Ellipsoidal(p = np.inf, 
                                                                  rho=2.))
n = 5

# formulate cvxpy variable
x_el = cp.Variable(n)

# formulate problem constants
A = np.diag(0.5*side)
b = center

# formulate objective
objective = cp.Minimize(-c@x_el)

# formulate constraints
constraints = [(P@A@ellip_u + P@b)@ x_el  <= 10+(2*A@ellip_u+ 2*b)@x_el, x_el >=0, x_el<=1]

# formulate Robust Problem
prob_robust = lro.RobustProblem(objective, constraints)

# solve
newprob = prob_robust.dualize()
newprob.solve()

print("Ellipsoidal objective value: ", newprob.objective.value, "\nEllipsoidal x: ", x_el.value)

# assert x values are equal
npt.assert_allclose(x_box.value, x_el.value, rtol=RTOL, atol=ATOL)


Box objective value:  -1.873180154775034 
Box x:  [1.00000000e+00 6.25519596e-01 9.99999999e-01 4.90558504e-11
 1.40880238e-10]
Ellipsoidal objective value:  -1.8731801547750322 
Ellipsoidal x:  [1.00000000e+00 6.25519596e-01 9.99999999e-01 4.90551238e-11
 1.40878693e-10]


We compare the above with its explicit reformulation and solution
$$
\begin{array}{ll}
\text{minimize} & c^Tx\\
\text{subject to}  & (Pb)^Tx - 2b^Tx + \rho\|A^TP^Tx - 2A^Tx\|_1 \leq  10,\\
\end{array}
$$

In [9]:
# formulate using cvxpy
x_cvxpy = cp.Variable(5)
# formulate objective
objective = cp.Minimize(-c@x_cvxpy)

# formulate constraints
constraints = [(P@b)@x_cvxpy - 2*b@x_cvxpy + 2*cp.norm((P@A).T@x_cvxpy - 2*A.T@x_cvxpy,p=1)<= 10, x_cvxpy>=0, x_cvxpy<=1]

# formulate problem
prob_cvxpy = cp.Problem(objective, constraints)

# solve
prob_cvxpy.solve()

print("Cvxpy objective value: ", prob_cvxpy.objective.value, "\nCvxpy x: ", x_cvxpy.value)

# assert x values are equal
npt.assert_allclose(x_box.value, x_cvxpy.value, rtol=RTOL, atol=ATOL)


Cvxpy objective value:  -1.8731801548079365 
Cvxpy x:  [1.00000000e+00 6.25519596e-01 9.99999999e-01 3.18587860e-11
 1.03709479e-10]


#### Example 3: Multiple uncertain constraints, polyhedral uncertainty
We solve the problem
$$
\begin{array}{ll}
\text{minimize} & c^Tx\\
\text{subject to}  & (P_1u + a)^Tx  \leq  10\\
& (P_2u)^Tx  \leq  5\\
& x \leq 5,
\end{array}
$$
where $c$, $P_1$, $P_2$, and $a$ are constants, and $u$ is the uncertain parameter from a polyhedral uncertainty set

In [10]:
#restate the ellipsoidal set 
D =  np.vstack((np.eye(m), -2*np.eye(m)))
d = np.concatenate((0.1*np.ones(m), 0.1*np.ones(m)))
poly_u = lro.UncertainParameter(m, 
                                uncertainty_set = lro.Polyhedral(
                                                        D = D, 
                                                        d = d))
n = 4
# formulate cvxpy variable
x_r = cp.Variable(n)

# formulate problem constants
P1 = 0.5 * np.eye(m)[:n, :]
P2 = 3*np.random.rand(n,m)
a = 0.1 * np.random.rand(n)
c = np.random.rand(n)

# formulate objective
objective = cp.Minimize(-c@x_r)

# formulate constraints
constraints = [(P1@poly_u +a)@ x_r <=10, x_r<=5]
constraints += [(P2@poly_u)@ x_r <= 5]

# formulate Robust Problem
prob_robust = lro.RobustProblem(objective, constraints)

# solve

new_prob = prob_robust.dualize()
new_prob.solve()
print("LRO objective value: ", new_prob.objective.value, "\nLRO x: ", x_r.value)


LRO objective value:  -4.71582850351676 
LRO x:  [-4.39416814  5.          5.          2.53800077]


We compare it with its direct reformulation, where each uncertain constraint is treated independently,
$$
\begin{array}{ll}
\text{minimize} & c^Tx\\
\text{subject to}  & a^Tx + w_1^Td  \leq  10\\
& w_1^TD = P_1^Tx\\
& w_2^Td  \leq  5\\
& w_2^TD = P_2^Tx\\
& w_1, w_2 \geq 0\\
& x \leq 5,
\end{array}
$$
where $w_1$, $w_2$ are new variables introduced.

In [11]:
# formulate using cvxpy
x_cvxpy = cp.Variable(n)
w1 = cp.Variable(2*m)
w2 = cp.Variable(2*m)
# formulate objective
objective = cp.Minimize(-c@x_cvxpy)

# formulate constraints
constraints = [a@x_cvxpy + w1@d <= 10]
constraints += [w1@D == P1.T@x_cvxpy]
constraints += [w2@d <= 5]
constraints += [w2@D == P2.T@x_cvxpy]
constraints += [w1>=0, w2 >= 0, x_cvxpy <=5]
# formulate Robust Problem
prob_cvxpy = cp.Problem(objective, constraints)

# solve
prob_cvxpy.solve()
print("Cvxpy objective value: ", prob_cvxpy.objective.value, "\nCvxpy x: ", x_cvxpy.value)

# assert x values are equal
npt.assert_allclose(x_r.value, x_cvxpy.value, rtol=RTOL, atol=ATOL)

Cvxpy objective value:  -4.7158285036491305 
Cvxpy x:  [-4.39416814  5.          5.          2.53800077]


#### Example 4: Direct comparison constraints, budget uncertainty
We solve the problem
$$
\begin{array}{ll}
\text{minimize} & c^Tx\\
\text{subject to}  & (Pu + a)^Tx  \leq  10\\
& x \geq P_1u,
\end{array}
$$
where $c$, $P$,and $a$ are constants, and $u$ is the uncertain parameter from a budet uncertainty set. 
We can include constraints that relates directly the values of $x$ and $u$, as in the second constraint.

In [12]:
#restate the ellipsoidal set
budget_u = lro.UncertainParameter(m, 
                                  uncertainty_set = lro.Budget(rho1=2., 
                                                               rho2 = 1., A1 = np.eye(m), A2 = np.eye(m)))
n = 4
# formulate cvxpy variable
x_r = cp.Variable(n)

# formulate problem constants
P = 3. * np.eye(m)[:n, :]
P1 = 3*np.random.rand(n,m)
a = 0.1 * np.random.rand(n)
c = np.random.rand(n)

# formulate objective
objective = cp.Minimize(c@x_r)

# formulate constraints
constraints = [(P@budget_u +a)@ x_r <= 10]
constraints += [x_r >= P1@budget_u]

# formulate Robust Problem
prob_robust = lro.RobustProblem(objective, constraints)

# solve
new_prob = prob_robust.dualize()
new_prob.solve()
print("LRO objective value: ", new_prob.objective.value, "\nLRO x: ", x_r.value)


LRO objective value:  8.603363276249897 
LRO x:  [2.23422537 2.97229215 2.91104176 2.35340157]


We compare this with its direct reformulation,
$$
\begin{array}{ll}
\text{minimize} & c^Tx\\
\text{subject to}  & a^Tx + \rho_1\|y\|_1 + \rho_2\|P^Tx - y\|_\infty  \leq  10\\
& -x_i + \rho_1\|z_i\|_1 + \rho_2\|(P_1^T)_i - z_i\|_\infty \leq 0, \quad i = 1,\dots,n
\end{array}
$$
where $(P_1^T)_i$ is the $i$'th row of $P_1$.

In [13]:
# formulate cvxpy variables
x_cvxpy = cp.Variable(n)
y = cp.Variable(m)
z = cp.Variable((n,m))

# formulate objective
objective = cp.Minimize(c@x_cvxpy)

# formulate constraints
constraints = [a@x_cvxpy + 2*cp.norm(y,1) + cp.norm(P.T@x_cvxpy-y,np.inf) <= 10]
for i in range(n):
  constraints += [-x_cvxpy[i] + 2*cp.norm(z[i],1) + cp.norm(P1.T@(np.eye(n)[i]) - z[i],np.inf) <= 0]

# formulate Robust Problem
prob_cvxpy = cp.Problem(objective, constraints)

# solve
prob_cvxpy.solve()
print("Cvxpy objective value: ", prob_cvxpy.objective.value, "\nCvxpy x: ", x_cvxpy.value)

# assert x values are equal
npt.assert_allclose(x_r.value, x_cvxpy.value, rtol=RTOL, atol=ATOL)


Cvxpy objective value:  8.60336327790219 
Cvxpy x:  [2.23422537 2.97229215 2.91104176 2.35340157]


#### Example 5: Quadratic uncertainty, ellipsoidal set
We solve the problem
$$
\begin{array}{ll}
\text{minimize} & \tau\\
\text{subject to}  & -(1/2)\sum_{i=1}^n (u^TP_iu)x_i  \leq  \tau\\
& \mathbf{1}^Tx = 3\\
& x \geq 0, \quad x \leq 1
\end{array}
$$
where each $P_i\in \mathbf{R}^{n\times n}$ is a symmetric positive definite matrix, and $u$ is the uncertain parameter from an ellipsoidal uncertainty set. 

In [14]:
# formulate uncertainty set
n = 5
u = lro.UncertainParameter(n,
                        uncertainty_set=lro.Ellipsoidal(p=2,
                                                    rho=0.5))
# formulate cvxpy variables
x_r = cp.Variable(n)
t = cp.Variable()

# formulate problem constants
P = {}
P_inv = {}
for i in range(n):
    P[i] = datasets.make_spd_matrix(n, random_state=i)
    P_inv[i] = sc.linalg.sqrtm(np.linalg.inv(P[i]))

# formulate objective
objective = cp.Minimize(t)

# formulate constraints 
constraints = [cp.sum([-0.5*lro.quad_form(u, P[i]*x_r[i]) for i in range(n)]) <= t]
constraints += [cp.sum(x_r)==4]
constraints += [x_r >= 0.6, x_r <= 1]

# formulate Robust Problem
prob_robust = lro.RobustProblem(objective, constraints)

new_prob = prob_robust.dualize()
new_prob.solve()
print("LRO objective value: ", new_prob.objective.value, "\nLRO x: ", x_r.value)

LRO objective value:  -2.001521732684513e-10 
LRO x:  [0.79877228 0.80135493 0.80078944 0.79878038 0.80030297]


We compare this with the direct reformulation,
\begin{equation}
	\begin{array}{lll}
		&\text{minimize} & f(x)\\
		&\text{subject to} & (1/2)\sum_{i=1}^n (z_i^TP_i^{-1}z_i)/x_i + \rho\|z\|_2 \le 0\\
		&& \sum_{i=1}^n z_i = z,\\
&& \mathbf{1}^Tx = 3\\
&& x \geq 0, \quad x \leq 1
	\end{array}
\end{equation}
where we have $\gamma \in \mathbf{R}^m, z \in \mathbf{R}^n, z_i \in \mathbf{R}^n$, $i = 1,\dots,n$, as auxiliary variables. 

In [15]:
# formulate using cvxpy
x_cvxpy = cp.Variable(n)
t = cp.Variable()
z = cp.Variable(n)
y = cp.Variable((n,n))
gam = cp.Variable(n)

# formulate objective
objective = cp.Minimize(t)

# formulate constraints
constraints = [cp.sum([cp.quad_over_lin(y[i]@P_inv[i],2*x_cvxpy[i]) for i in range(n)]) + 0.5*cp.norm(z,2) <= t]
constraints += [cp.sum(y, axis = 1) == z]
constraints += [cp.sum(x_cvxpy) == 4]
constraints += [x_cvxpy >= 0.6, x_cvxpy <= 1]

# formulate problem 
prob_cvxpy = cp.Problem(objective, constraints)
prob_cvxpy.solve()
print("Cvxpy objective value: ", prob_cvxpy.objective.value, "\nCvxpy x: ", x_cvxpy.value)

# assert x values are equal
npt.assert_allclose(x_r.value, x_cvxpy.value, rtol=RTOL, atol=ATOL)


Cvxpy objective value:  -2.1195914161480326e-10 
Cvxpy x:  [0.79880041 0.80132141 0.80077192 0.79880835 0.80029791]


#### Example 6: Max of affine uncertainty, ellipsoidal set
We solve the problem
$$
\begin{array}{ll}
		\text{minimize} & \tau \\
		\text{subject to } & \max\{a^Tx - d^Tx, a^Tx - d^T(3u+3)\} \leq \tau,\\
\end{array}
$$
where $a$ and $d$ are constants, and $u$ is the uncertain parameter from an ellipsoidal uncertainty set. 

In [16]:
# formulate uncertainty set
n = 5
u = lro.UncertainParameter(n,
                        uncertainty_set=lro.Ellipsoidal(p=2,
                                                    rho=1))
# formulate cvxpy variables
x_r = cp.Variable(n)
t = cp.Variable()

# formulate constants
a = npr.uniform(1,4,n)
d = a + npr.uniform(2,5,n)

# formulate objective
objective = cp.Minimize(t)

# formulate constraints 
constraints = [cp.maximum(a@x_r - d@x_r, a@x_r - d@(3*u+3)) <= t]
constraints += [x_r >= 0]


# formulate Robust Problem
prob_robust = lro.RobustProblem(objective, constraints)

new_prob = prob_robust.dualize()
new_prob.solve()
print("LRO objective value: ", new_prob.objective.value, "\nLRO x: ", x_r.value)

LRO objective value:  -21.647219476830386 
LRO x:  [5.83434195e-08 1.95663034e-10 2.20657553e-11 1.31625990e+01
 4.34986404e-11]


In [17]:
# formulate using cvxpy
x_cvxpy = cp.Variable(n)
t = cp.Variable()

# formulate objective
objective = cp.Minimize(t)

# formulate constraints
constraints = [a@x_cvxpy -d@x_cvxpy + cp.norm(3*d,2) <= t]
constraints += [a@x_cvxpy -3*d@np.ones(n) + cp.norm(3*d,2) <= t]
constraints += [x_cvxpy >= 0]

# formulate problem 
prob_cvxpy = cp.Problem(objective, constraints)
prob_cvxpy.solve()
print("Cvxpy objective value: ", prob_cvxpy.objective.value, "\nCvxpy x: ", x_cvxpy.value)

# assert x values are equal
npt.assert_allclose(x_r.value, x_cvxpy.value, rtol=RTOL, atol=ATOL)


Cvxpy objective value:  -21.647219445869396 
Cvxpy x:  [2.62399952e-06 9.47772829e-10 1.41821592e-09 1.31625975e+01
 1.02770417e-09]
